In [2]:
import os,re
os.listdir()

['.ipynb_checkpoints',
 'email_word-prediction.ipynb',
 'krishan_sent_outlook.CSV']

In [3]:
valid_lines = []


include = False
for line in open('krishan_sent_outlook.CSV','r'):
    if line.startswith('From: Krishan'):
        include=True
        continue
    elif line.startswith('From:') or line.startswith('Thanks and Regards'):
        include = False
        continue
    if include:
        if re.match(r'\w',line) and not re.match(r'.+:',line) and len(line.split())>=3 and re.match(r'[a-zA-Z]',line):
            valid_lines.append(line.strip().lower())

valid_lines = [ re.sub('[^A-Za-z0-9]+', ' ', line).strip() for line in valid_lines ]
valid_lines[-100:]
len(valid_lines)

3641

In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(1000)
tokenizer.fit_on_texts(valid_lines)

def partitionlines(lines,maxlen):
    data_x=[]
    data_y=[]
    for line in lines:
        words = line
        for i in range (0,maxlen):
            if i >=len(words)-1:
                break
            data_x.append(words[0:i+1])
            data_y.append(words[i+1])
        for i in range (maxlen,len(words)):
            if i >=len(words)-1:
                break
            data_x.append(words[i-maxlen+1:i+1])
            data_y.append(words[i+1])
    return data_x,data_y
#partitionlines(['hello world every one . sunny day'],4)

maxlen=5
lines = tokenizer.texts_to_sequences(valid_lines)
data_x,data_y = partitionlines(lines,maxlen)
len(data_x)

32283

In [5]:
tokenizer.word_index['the']

1

In [6]:
len(data_x)

32283

In [8]:
#print(x_train[2891],np.argmax(y_train[2891]))
#print(x_train_shuffled[0],np.argmax(y_train_shuffled[0]))

In [118]:
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(data_x,maxlen=maxlen)
from keras.utils import to_categorical
y_train = to_categorical(data_y)

data_y[0]
np.argmax(y_train[5])
len(y_train[5])
max(data_y)
y_train.shape

(32283, 1000)

In [10]:
import numpy as np
np.max(y_train,axis=1)
#shuffle
indices = np.arange(0,len(x_train))
np.random.shuffle(indices)
x_train_shuffled = x_train[indices]
y_train_shuffled= y_train[indices]
indices

array([25292, 16463,  7883, ...,  3105,   342, 16743])

In [11]:
#split
x_test = x_train_shuffled[-1000:]
x_train_partial = x_train_shuffled[:-1000]

y_test = y_train_shuffled[-1000:]
y_train_partial = y_train_shuffled[:-1000]

In [18]:
from keras.models import Sequential
from keras import layers
from keras import losses
from keras import metrics

def getmodel():
    model = Sequential()
    model.add(layers.Embedding(1000, 32))
    model.add(layers.LSTM(32))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer='adam',loss=losses.categorical_crossentropy,metrics=[metrics.categorical_accuracy])

    model.summary()
    return model

In [20]:
model = getmodel()
history = model.fit(x_train_partial,
          y_train_partial,
          epochs=100,
          batch_size=32,
          validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          32000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              33000     
Total params: 73,320
Trainable params: 73,320
Non-trainable params: 0
_________________________________________________________________
Train on 25026 samples, validate on 6257 samples
Epoch 1/100
25026/25026 [==============================] - 13s 501us/step - loss: 6.1051 - categorical_accuracy: 0.0358 - val_loss: 5.8886 - val_categorical_accuracy: 0.0408
Epoch 2/100
25026/25026 [==============================] - 12s 471us/step - loss: 5.7057 - categorical_accuracy: 0.0431 - val_loss: 5.5349 - val_categorical_accuracy: 0.0564
Epoch

25026/25026 [==============================] - 12s 465us/step - loss: 1.2754 - categorical_accuracy: 0.7354 - val_loss: 1.9697 - val_categorical_accuracy: 0.6620
Epoch 43/100
25026/25026 [==============================] - 12s 462us/step - loss: 1.2503 - categorical_accuracy: 0.7394 - val_loss: 1.9487 - val_categorical_accuracy: 0.6620
Epoch 44/100
25026/25026 [==============================] - 12s 485us/step - loss: 1.2265 - categorical_accuracy: 0.7427 - val_loss: 1.9330 - val_categorical_accuracy: 0.6706
Epoch 45/100
25026/25026 [==============================] - 12s 476us/step - loss: 1.2047 - categorical_accuracy: 0.7475 - val_loss: 1.9265 - val_categorical_accuracy: 0.6692
Epoch 46/100
25026/25026 [==============================] - 12s 468us/step - loss: 1.1857 - categorical_accuracy: 0.7501 - val_loss: 1.9136 - val_categorical_accuracy: 0.6783
Epoch 47/100
25026/25026 [==============================] - 11s 445us/step - loss: 1.1655 - categorical_accuracy: 0.7529 - val_loss: 1.899

Epoch 89/100
25026/25026 [==============================] - 16s 642us/step - loss: 0.6829 - categorical_accuracy: 0.8438 - val_loss: 1.6747 - val_categorical_accuracy: 0.7516
Epoch 90/100
25026/25026 [==============================] - 13s 526us/step - loss: 0.6712 - categorical_accuracy: 0.8454 - val_loss: 1.6744 - val_categorical_accuracy: 0.7564
Epoch 91/100
25026/25026 [==============================] - 12s 486us/step - loss: 0.6704 - categorical_accuracy: 0.8463 - val_loss: 1.6727 - val_categorical_accuracy: 0.7548
Epoch 92/100
25026/25026 [==============================] - 12s 481us/step - loss: 0.6652 - categorical_accuracy: 0.8468 - val_loss: 1.6532 - val_categorical_accuracy: 0.7590
Epoch 93/100
25026/25026 [==============================] - 13s 515us/step - loss: 0.6536 - categorical_accuracy: 0.8488 - val_loss: 1.6588 - val_categorical_accuracy: 0.7588
Epoch 94/100
25026/25026 [==============================] - 12s 460us/step - loss: 0.6477 - categorical_accuracy: 0.8503 - va

In [1]:
reverse_word_index = [index:word for word,index in tokenizer.word_index]
def predict(input_lines):
    input_lines = [line.strip().lower() + ' end' for line in input_lines]
    lines = tokenizer.texts_to_sequences(input_lines)
    input_x,input_y = partitionlines(lines,maxlen)
    model_input_x = sequence.pad_sequences(input_x,maxlen=maxlen)
    output_y = model.predict(model_input_x)
    output_seq = np.argmax(output_y,axis = 0)
    output_lines = [[reverse_word_index[num] for num in inp] for inp in input_x]
    predicted_words = [reverse_word_index[num] for num in output_seq]
    expected_words = [reverse_word_index[num] for num in input_y]
    for i in range(0,len(input_lines)):
        print ("input = :{0}, predicted = {1},expected = {2}", output_lines[i],predicted_words[i],expected_words[i])
    

Using TensorFlow backend.


In [79]:
reverse_word_index = {index:word for word,index in tokenizer.word_index.items()}
def wordfromonehot(onehot):
    index = np.argmax(onehot)
    return reverse_word_index[index]
def predict(input_line, count ):
    input_line_clean = input_line.strip().lower()
    #print(input_lines)
    predicted_words = []
    input_x= tokenizer.texts_to_sequences([input_line_clean])[0]
    for _ in range(0,count):
        model_input_x = sequence.pad_sequences([input_x],maxlen=maxlen)
        output_y = model.predict(model_input_x)
        #print (np.max(output_y))
        input_x.append(np.argmax(output_y))
        predicted_words.append((wordfromonehot(output_y), np.max(output_y)))
    #print(predicted_words)
    print ("input = {0}, predicted = {1}".format(input_line,predicted_words))

In [80]:
predict("Can you please",10)

input = Can you please, predicted = [('check', 0.23085266), ('the', 0.35568327), ('quality', 0.8381603), ('and', 0.9865416), ('let', 0.96286523), ('me', 0.9975465), ('know', 0.99354774), ('your', 0.7261728), ('feedback', 0.34738642), ('and', 0.851428)]


In [95]:
model.evaluate(x_test,y_test)

1000/1000 [==============================] - 0s 94us/step


[1.7531049308776856, 0.753]

In [ ]:
x_test_words = []
for seq in x_test:
    x_test_words.append(' '.join([reverse_word_index[num] for num in seq if num != 0 ]))
pred_test = model.predict(x_test)

y_test_pred_words = [wordfromonehot(pred) for pred in pred_test]
y_act = [wordfromonehot(y) for y in y_test]
for i in range(100,110):
    #print('input = {0}, pred  = {1}, actual = {2}'.format(x_test_words[i],y_test_pred_words[i],y_act[i]))
    predict(x_test_words[i],4)

In [135]:
model.save('outlook_pred.h5')